In [1]:
# Data Wrangling
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# PreProcessing Data
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

#Geocoding
import requests
import geopandas as gpd
import geopy
import folium
from folium import plugins

In [2]:
EV_data = pd.read_csv("E:\Deakin\Cleaned_Australian_EV_Charging_Stations.csv")

In [3]:
EV_data.columns

Index(['Unnamed: 0', 'Location Name', 'Latitude', 'Longitude', 'Town',
       'Postal Code', 'City', 'Address', 'Plugs_Type2', 'Plugs_Three_Phase',
       'Plugs_CHAdeMO', 'Plugs_CCS/SAE', 'Plugs_Tesla', 'Plugs_J-1772',
       'Plugs_Caravan_Mains_Socket', 'Plugs_wall_AU/NZ', 'Power 1',
       'charging_stations', 'Nearby EVStations', 'Hospitals', 'Parks',
       'Restaurants', 'Malls', 'Supermarkets'],
      dtype='object')

In [4]:
EV_geo_data = EV_data.drop(columns = {'Unnamed: 0','Plugs_Type2', 'Plugs_Three_Phase',
       'Plugs_CHAdeMO', 'Plugs_CCS/SAE', 'Plugs_Tesla', 'Plugs_J-1772',
       'Plugs_Caravan_Mains_Socket', 'Plugs_wall_AU/NZ', 'Power 1',
       'charging_stations','Hospitals', 'Parks',
       'Restaurants', 'Malls', 'Supermarkets'})

In [5]:
EV_geo_data

,Location Name,Latitude,Longitude,Town,Postal Code,City,Address,Nearby EVStations
0,Chargefox Charging Station,-35.726720,145.659354,Cobram,2714,N.A,154 Campbell Rd,0
1,Secure Parking,-37.832108,144.969126,N.A,3205,Melbourne,"15-29 Bank Street, South Melbourne VIC 3205, A...",15
2,Lonsdale St,-37.813437,144.955934,N.A,3000,Melbourne,"535 Little Lonsdale St, Melbourne VIC 3004, Au...",20
3,Sorrento Community Centre,-38.338560,144.737043,N.A,3943,Melbourne,"860 Melbourne Rd, Sorrento VIC 3943, Australia",1
4,CTR Whittlesea,-37.510590,145.118412,N.A,3757,Melbourne,"74A Church St, Whittlesea VIC 3757, Australia",1
...,...,...,...,...,...,...,...,...
386,Brimbank Shopping Centre,-37.751827,144.775257,N.A,3023,Melbourne,"Brimbank Central Shopping Centre shop 81t, 28-...",1
387,QV Melbourne - Car Park,-37.810679,144.966601,N.A,3000,Melbourne,"180 Lonsdale Street, Melbourne VIC 3000, Austr...",16
388,Carlton,-37.803202,144.971757,N.A,3053,Melbourne,"11 Nicholson St, Carlton VIC 3053, Australia",16
389,RACV Solar,-38.197552,144.344603,N.A,3216,Geelong,"Units 5 & 6, 8 Lewalan St, Grovedale VIC 3216,...",2


In [6]:
EV_geo_data['City'].value_counts()

Melbourne          299
N.A                 65
Geelong              7
Cairns               6
Bendigo              3
Townsville           2
Shepparton           2
Brisbane City        1
Ballarat             1
Rockhampton          1
Maryborough          1
Mackay               1
Toowoomba            1
Gold Coast City      1
Name: City, dtype: int64

In [7]:
import requests

def get_directions_response(lat1, long1, lat2, long2, mode='drive'):
    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    key = "1d66cca7a7msh2a3c2781896706bp150701jsn98546104fda2"
    host = "route-and-directions.p.rapidapi.com"
    headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
    querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response

In [8]:
response = get_directions_response(EV_geo_data['Latitude'][0],EV_geo_data['Longitude'][0] , EV_geo_data['Latitude'][1],EV_geo_data['Longitude'][1])

In [9]:
response.json()

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [145.659354, -35.726719], 'original_index': 0},
     {'location': [144.969126, -37.832108], 'original_index': 1}],
    'units': 'metric',
    'distance': 303896,
    'distance_units': 'meters',
    'time': 11596.603,
    'legs': [{'distance': 303896,
      'time': 11596.603,
      'steps': [{'from_index': 0,
        'to_index': 2,
        'distance': 1246,
        'time': 224.452,
        'instruction': {'text': 'Drive west on Mardenoora Road.'}},
       {'from_index': 2,
        'to_index': 5,
        'distance': 1588,
        'time': 119.611,
        'instruction': {'text': 'Turn right onto Woolshed Road.'}},
       {'from_index': 5,
        'to_index': 21,
        'distance': 9430,
        'time': 528.166,
        'instruction': {'text': 'Turn left onto Fullers Road.'}},
       {'from_index': 21,
        'to_index': 92,
        'distance': 14355,
        'time': 557.115,
        'inst

In [10]:
distance = response.json()['features'][0]['properties']['distance']
distance_KM = round(distance/1000,2)
print(distance_KM)
time = response.json()['features'][0]['properties']['time']
time_hr = round((time/60)/60,2)
print(time_hr)


303.9
3.22


In [11]:
distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(distance_KM)+" Km </strong>" +"</h4></b>"
duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(time_hr)+" Hours. </strong>" +"</h4></b>"

In [12]:
response.json()['features'][0]['geometry']['coordinates']

[[[145.658249, -35.732336],
  [145.649507, -35.731203],
  [145.644622, -35.730554],
  [145.647086, -35.718037],
  [145.647131, -35.717823],
  [145.647389, -35.71646],
  [145.637775, -35.715267],
  [145.633701, -35.714844],
  [145.632859, -35.714741],
  [145.632534, -35.714671],
  [145.612536, -35.709364],
  [145.61108, -35.708989],
  [145.61098, -35.708963],
  [145.610726, -35.708896],
  [145.605579, -35.707522],
  [145.601545, -35.706474],
  [145.590896, -35.703636],
  [145.580432, -35.700848],
  [145.580176, -35.70078],
  [145.578734, -35.700398],
  [145.578488, -35.700333],
  [145.547638, -35.692156],
  [145.542462, -35.702138],
  [145.542421, -35.702218],
  [145.542258, -35.702531],
  [145.538609, -35.709471],
  [145.535355, -35.715656],
  [145.533234, -35.71969],
  [145.532655, -35.721224],
  [145.532271, -35.724095],
  [145.531739, -35.729088],
  [145.531355, -35.734431],
  [145.531404, -35.735065],
  [145.531625, -35.737942],
  [145.531895, -35.739719],
  [145.532739, -35.741618

In [13]:
coordinates = []
for _, station in EV_geo_data.iterrows():
    location = [[station["Latitude"], station["Longitude"]]]
    coordinates += location

In [14]:
EV_geo_data['coordinates'] = coordinates
EV_geo_data

,Location Name,Latitude,Longitude,Town,Postal Code,City,Address,Nearby EVStations,coordinates
0,Chargefox Charging Station,-35.726720,145.659354,Cobram,2714,N.A,154 Campbell Rd,0,"[-35.726719565595815, 145.65935410367524]"
1,Secure Parking,-37.832108,144.969126,N.A,3205,Melbourne,"15-29 Bank Street, South Melbourne VIC 3205, A...",15,"[-37.832108, 144.969126]"
2,Lonsdale St,-37.813437,144.955934,N.A,3000,Melbourne,"535 Little Lonsdale St, Melbourne VIC 3004, Au...",20,"[-37.81343708, 144.955934]"
3,Sorrento Community Centre,-38.338560,144.737043,N.A,3943,Melbourne,"860 Melbourne Rd, Sorrento VIC 3943, Australia",1,"[-38.33856, 144.737043]"
4,CTR Whittlesea,-37.510590,145.118412,N.A,3757,Melbourne,"74A Church St, Whittlesea VIC 3757, Australia",1,"[-37.51059, 145.118412]"
...,...,...,...,...,...,...,...,...,...
386,Brimbank Shopping Centre,-37.751827,144.775257,N.A,3023,Melbourne,"Brimbank Central Shopping Centre shop 81t, 28-...",1,"[-37.751827, 144.775257]"
387,QV Melbourne - Car Park,-37.810679,144.966601,N.A,3000,Melbourne,"180 Lonsdale Street, Melbourne VIC 3000, Austr...",16,"[-37.810679, 144.966601]"
388,Carlton,-37.803202,144.971757,N.A,3053,Melbourne,"11 Nicholson St, Carlton VIC 3053, Australia",16,"[-37.80320151, 144.9717566]"
389,RACV Solar,-38.197552,144.344603,N.A,3216,Geelong,"Units 5 & 6, 8 Lewalan St, Grovedale VIC 3216,...",2,"[-38.197552, 144.344603]"


In [15]:
geo = response.json()['features'][0]['geometry']['coordinates'][0]
geo_df = pd.DataFrame (geo, columns = ['long','lat'])

coordinates_res = []
for _, station in geo_df.iterrows():
    location = [[station["lat"], station["long"]]]
    coordinates_res += location

In [16]:
from geopy.distance import distance

dist = []
position_J = []
position_i = []
for i in range(len(coordinates_res)):
    for j in range(len(coordinates)):
        if i == j:
            pass
        else:
            if distance((coordinates_res[i]),(coordinates[j])) < 5:
                                                  dist.append(distance((coordinates_res[i]),(coordinates[j])))
                                                  position_i.append(coordinates_res[i])
                                                  position_J.append(coordinates[j])
                        

In [17]:
position_J

[[-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-35.726719565595815, 145.65935410367524],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38578464084992, 145.3969652903047],
 [-36.38

In [18]:
final_cords = []

for item in position_J: 
    if item not in final_cords: 
        final_cords.append(item) 
final_cords

[[-35.726719565595815, 145.65935410367524],
 [-36.38578464084992, 145.3969652903047],
 [-37.565285, 144.893834],
 [-37.616294, 144.914873],
 [-37.648097, 144.994195],
 [-37.649756, 145.008102],
 [-37.653416, 145.016892],
 [-37.696652, 144.959707],
 [-37.70191464, 144.9681533],
 [-37.702614, 144.969502],
 [-37.718952, 144.961498],
 [-37.71912127, 144.9615062],
 [-37.708497, 144.924037],
 [-37.723715, 144.920974],
 [-37.720262, 144.895849],
 [-37.719767, 144.89451],
 [-37.719978, 144.892935],
 [-37.71395622, 144.8860831],
 [-37.713651, 144.885527],
 [-37.709632, 144.871383],
 [-37.684086, 144.863519],
 [-37.735219, 144.887807],
 [-37.772002, 144.887869],
 [-37.76556152, 144.9236774],
 [-37.765834, 144.923586],
 [-37.736669, 144.969336],
 [-37.74058156, 144.9694655],
 [-37.74111, 144.970149],
 [-37.771404, 144.962021],
 [-37.77360197, 144.9618831],
 [-37.773851, 144.961886],
 [-37.79687512, 144.926417],
 [-37.797009, 144.926387],
 [-37.802158, 144.96708],
 [-37.80216073, 144.9671638],
 [-

In [19]:
EV_geo_final = EV_geo_data[EV_geo_data['coordinates'].isin(final_cords)]

In [20]:
EV_geo_final

,Location Name,Latitude,Longitude,Town,Postal Code,City,Address,Nearby EVStations,coordinates
0,Chargefox Charging Station,-35.726720,145.659354,Cobram,2714,N.A,154 Campbell Rd,0,"[-35.726719565595815, 145.65935410367524]"
1,Secure Parking,-37.832108,144.969126,N.A,3205,Melbourne,"15-29 Bank Street, South Melbourne VIC 3205, A...",15,"[-37.832108, 144.969126]"
2,Lonsdale St,-37.813437,144.955934,N.A,3000,Melbourne,"535 Little Lonsdale St, Melbourne VIC 3004, Au...",20,"[-37.81343708, 144.955934]"
9,Melbourne Museum,-37.804609,144.972702,N.A,N.A,Melbourne,"11 Nicholson St, Carlton VIC 3053, Australia",17,"[-37.804609, 144.972702]"
11,Club Trillium (gym/pool) carpark,-37.565285,144.893834,N.A,3064,Melbourne,"Trillium Boulevard, Mickleham VIC 3064, Australia",0,"[-37.565285, 144.893834]"
...,...,...,...,...,...,...,...,...,...
378,Kensington,-37.796875,144.926417,N.A,3031,Melbourne,"72 Altona Street, Kensington VIC 3031",3,"[-37.79687512, 144.926417]"
381,Flinders St,-37.818032,144.964010,N.A,3000,Melbourne,"300 Flinders St, Melbourne VIC 3000, Australia",17,"[-37.8180318, 144.9640101]"
384,Moonee Ponds Supercharger,-37.765834,144.923586,N.A,3039,Melbourne,"665 Mt Alexander Rd, Moonee Ponds VIC 3039, Au...",4,"[-37.765834, 144.923586]"
387,QV Melbourne - Car Park,-37.810679,144.966601,N.A,3000,Melbourne,"180 Lonsdale Street, Melbourne VIC 3000, Austr...",16,"[-37.810679, 144.966601]"


In [23]:
def create_map(response,loc1,loc2):
   # use the response

   geo = response.json()['features'][0]['geometry']['coordinates']
   points = [(i[1], i[0]) for i in geo[0]]
   geomap = folium.Map()
   # add marker for the start and ending points
   for point in [points[0], points[-1]]:
        folium.Marker(point, icon = None).add_to(geomap)
        distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(distance_KM)+" Km </strong>" +"</h4></b>"
        duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(time_hr)+" Hr </strong>" +"</h4></b>"
        
        folium.PolyLine(points, weight=5, opacity=1 , popup = distance_txt+duration_txt).add_to(geomap)
        df = pd.DataFrame(geo[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
        start = df[['Lat', 'Lon']].min().values.tolist()
        end = df[['Lat', 'Lon']].max().values.tolist()
        geomap.fit_bounds([start, end])
        folium.Marker(location = points[0], icon=folium.Icon(color="green"), popup = loc1).add_to(geomap)
        folium.Marker(location= points[-1],icon=folium.Icon(color="red") , popup = loc2 ).add_to(geomap)
        
        for _, station in EV_geo_final.iterrows():
            folium.Marker(location = [station["Latitude"], station["Longitude"]],
                          popup = station["Location Name"] +  station["Address"],
                          tooltip = station["Location Name"] , icon = folium.Icon(icon = 'flash',color = 'green')).add_to(geomap)
        return geomap

In [24]:
geomap = create_map(response,EV_geo_data['Location Name'][0],EV_geo_data['Location Name'][1])
geomap